In [ ]:
from cosipy import response
from cosipy import test_data
from cosipy import BinnedData
from cosipy import DetectorResponse
from histpy import Histogram, Axes, Axis, HealpixAxis
import astropy.units as u
import numpy as np
import healpy as hp

In [ ]:
full_detector_response = response.FullDetectorResponse.open(test_data.path / "test_full_detector_response.h5")
full_detector_response

# Binned Histogram

In [ ]:
analysis = BinnedData(test_data.path / "inputs_crab.yaml")
analysis

In [ ]:
analysis.energy_bins = full_detector_response.axes['Em'].edges.to(u.keV).value
analysis.nside = full_detector_response.axes['PsiChi'].nside

assert np.all(full_detector_response.axes['Phi'].widths == full_detector_response.axes['Phi'].widths[0]) == True

analysis.phi_pix_size = full_detector_response.axes['Phi'].widths[0].to(u.deg).value

analysis.time_bins = 10 #s

In [ ]:
analysis.get_binned_data(test_data.path / "unbinned_data_MEGAlib_calc.hdf5", output_name = "test_event_histogram_localCDS", psichi_binning = 'local')

In [ ]:
analysis.binned_data.axes['Time'].edges

In [ ]:
np.all(analysis.binned_data.axes['Em'].edges == full_detector_response.axes['Em'].edges)

In [ ]:
# note that axis_scale of the response and event histogram are not consistent (2024-07-04)

print(analysis.binned_data.axes['Em'] == full_detector_response.axes['Em'])
print(analysis.binned_data.axes['Em'].axis_scale, full_detector_response.axes['Em'].axis_scale)

In [ ]:
analysis.binned_data.axes['Phi'] == full_detector_response.axes['Phi']

In [ ]:
analysis.binned_data.axes['PsiChi'] == full_detector_response.axes['PsiChi']

In [ ]:
analysis.get_binned_data("unbinned_data_MEGAlib_calc.hdf5", output_name = "test_event_histogram_galacticCDS", psichi_binning = 'galactic')

In [ ]:
# coordsys becomes different

print(analysis.binned_data.axes['PsiChi'].coordsys, full_detector_response.axes['PsiChi'].coordsys)
analysis.binned_data.axes['PsiChi'] == full_detector_response.axes['PsiChi']

In [ ]:
analysis.binned_data.axes['Time'].edges
# should it be a list of astropy.time.Time?

# pre-computed response file (for testing)

**the response is not rotated correctly because this is used just for unit test.**

In [ ]:
rsp_axes = [HealpixAxis(nside=1, scheme='ring', coordsys='galactic', label = 'NuLambda'),
            full_detector_response.axes['Ei'], full_detector_response.axes['Em'], full_detector_response.axes['Phi'],
            HealpixAxis(nside=1, scheme='ring', coordsys='galactic', label = 'PsiChi')]
    
precomputed_response = DetectorResponse(rsp_axes, unit = full_detector_response.unit * u.s)

In [ ]:
exposure_time = 10 * u.s

In [ ]:
precomputed_response.axes['PsiChi'] == analysis.binned_data.axes['PsiChi']

In [ ]:
precomputed_response.unit

In [ ]:
for ipix in range(hp.nside2npix(full_detector_response.nside)):
    precomputed_response[ipix] = full_detector_response[ipix].project(['Ei', 'Em', 'Phi', 'PsiChi']).to_dense().contents * exposure_time

In [ ]:
precomputed_response.write("test_precomputed_response.h5", overwrite=True)